In [2]:
import fastf1 as ff1
from fastf1 import plotting
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd
import os

In [6]:
plotting.setup_mpl()

# Define the cache directory
cache_dir = 'cache_dir'

# Create the cache directory if it doesn't exist
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

# Enable the cache
ff1.Cache.enable_cache(cache_dir)

pd.options.mode.chained_assignment = None

## Collect the data

In [16]:
# Get the schedule for the 2023 season
race = ff1.get_session(2023, "Australia", "R")
race.load()

core           INFO 	Loading data for Australian Grand Prix - Race [v3.3.8]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No

In [17]:
# Get the session laps
laps = race.laps

In [18]:
# Pick race drivers
laps_ric = laps.pick_driver('RIC')
laps_ver = laps.pick_driver('VER')
laps_ham = laps.pick_driver('HAM')
laps_bot = laps.pick_driver('BOT')
laps_nor = laps.pick_driver('NOR')
laps_sai = laps.pick_driver('SAI')
laps_yuk = laps.pick_driver('YUK')
laps_osc = laps.pick_driver('OSC')
laps_lec = laps.pick_driver('LEC')

In [19]:
# Analyse stint 1
laps_ric = laps_ric.loc[laps_ric["Stint"] == 1]
laps_ver = laps_ver.loc[laps_ver["Stint"] == 1]
laps_ham = laps_ham.loc[laps_ham["Stint"] == 1]
laps_bot = laps_bot.loc[laps_bot["Stint"] == 1]
laps_nor = laps_nor.loc[laps_nor["Stint"] == 1]
laps_sai = laps_sai.loc[laps_sai["Stint"] == 1]
laps_yuk = laps_yuk.loc[laps_yuk["Stint"] == 1]
laps_osc = laps_osc.loc[laps_osc["Stint"] == 1]
laps_lec = laps_lec.loc[laps_lec["Stint"] == 1]

In [20]:
# Expand the data
laps_ric["RaceLapNumber"] = laps_ric["LapNumber"] - 1
laps_ver["RaceLapNumber"] = laps_ver["LapNumber"] - 1
laps_ham["RaceLapNumber"] = laps_ham["LapNumber"] - 1
laps_bot["RaceLapNumber"] = laps_bot["LapNumber"] - 1
laps_nor["RaceLapNumber"] = laps_nor["LapNumber"] - 1
laps_sai["RaceLapNumber"] = laps_sai["LapNumber"] - 1
laps_yuk["RaceLapNumber"] = laps_yuk["LapNumber"] - 1
laps_osc["RaceLapNumber"] = laps_osc["LapNumber"] - 1
laps_lec["RaceLapNumber"] = laps_lec["LapNumber"] - 1

In [25]:
full_distance_ver_ric = pd.DataFrame()
full_distance_ver_ric = pd.DataFrame()

In [27]:
for lap in laps_ric.interlaps():
    telementary = lap[1].get_car_data().add_distance().add_driver_ahead()

    telementary = telementary.loc[telementary["DriverAhead"] == "3"]

    if len(telementary) != 0:
        lap_telementary = telementary[["Distance", "DistanceToDriverAhead"]]
        lap_telementary.loc[:, "Lap"] = lap[0] + 1

        full_distance_ver_ric = full_distance_ver_ric.append(lap_telementary)

        #Average/median distance
        average_distance = lap_telementary["DistanceToDriverAhead"].mean()
        median_distance = lap_telementary["DistanceToDriverAhead"].median()

        summarized_distance_ver_ric = summarized_distance_ver_ric.append({
            'Lap': lap[0] + 1,
            'Mean': average_distance,
            'Median': median_distance
        }, ignore_index = True)

AttributeError: 'Laps' object has no attribute 'interlaps'